<a href="https://colab.research.google.com/github/MohamedMadhoun/medical-rag-assistant/blob/main/Medical_RAG_Assistant_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#   Medical RAG Assistant 🩺


### A Retrieval-Augmented System for Symptom-Based Disease Prediction

### Setup and Installation

In [ ]:
import importlib.util
import os
import sys

# List of all packages we need
packages = ['langchain', 'langgraph', 'langchain-community', 'langchain-huggingface', 'faiss-cpu', 'beautifulsoup4', 'bitsandbytes', 'pandas']
all_installed = True

def is_installed(package_name):
    spec = importlib.util.find_spec(package_name)
    return spec is not None

# Loop through the packages and check if any are missing
for package in packages:
    if not is_installed(package):
        all_installed = False
        break

if not all_installed:
    print("Installing necessary packages...")
    !pip install langchain langgraph langchain-community langchain-huggingface faiss-cpu beautifulsoup4 bitsandbytes pandas -U -q
else:
    print("Packages already installed.")

# Define a flag file path
restart_flag_file = "/tmp/restart_flag"

# Check if the flag file exists (for Colab environments)
if not os.path.exists(restart_flag_file) and not all_installed:
    print("Restarting runtime to ensure correct library versions are loaded...")
    # Create the flag file
    with open(restart_flag_file, "w") as f:
        f.write("restarted")
    # This will restart the runtime
    os.kill(os.getpid(), 9)
else:
    print("Runtime ready.")

Installing necessary packages...
Runtime ready.


### Data Processing and Index Building

In [ ]:
# Code Cell 2: Load, Chunk Convert each row of merged data into document), and Index Documents from CSVs

import pandas as pd
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from typing import List

print("Loading, processing, and preparing medical knowledge base from CSVs...")

# 1. Uploading Files

df_dataset = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df_precaution = pd.read_csv('/content/drive/MyDrive/symptom_precaution.csv')
df_description = pd.read_csv('/content/drive/MyDrive/symptom_Description.csv')

# --- A. Processing the symptom file (dataset.csv) ---

# Fill in all empty values ​​in symptom columns with an empty text string ('')
symptom_cols = [col for col in df_dataset.columns if col.startswith('Symptom_')]
df_dataset[symptom_cols] = df_dataset[symptom_cols].fillna('')

# Combine all symptoms into one column for each record, removing spaces
def combine_symptoms(row):
# We only collect non-empty symptoms
    symptoms = [s.strip().replace('_', ' ') for s in row[symptom_cols] if s.strip()]
# Removing repeated symptoms (if the symptoms recur in the same class)
    unique_symptoms = list(set(symptoms))
    return ', '.join(unique_symptoms)

df_dataset['All_Symptoms'] = df_dataset.apply(combine_symptoms, axis=1)

# Remove the original columns for symptoms (Symptom_1, Symptom_2, ...)
df_dataset = df_dataset.drop(columns=symptom_cols)

# Removing duplicate rows containing the same disease and the same combined symptoms
df_dataset = df_dataset.drop_duplicates(subset=['Disease', 'All_Symptoms']).reset_index(drop=True)


# --- B. Processing the precaution file (symptom_precaution.csv) ---


# Fill in the empty values ​​in the precautionary columns with a RAG-use string
precaution_cols = [col for col in df_precaution.columns if col.startswith('Precaution_')]
df_precaution[precaution_cols] = df_precaution[precaution_cols].fillna('No Further Precaution Mentioned')

# Combine precautions into one column for each disease
def combine_precautions(row):
    precautions = [p.strip() for p in row[precaution_cols] if p.strip() != 'No Further Precaution Mentioned']
    return ' | '.join(precautions)

df_precaution['All_Precaution'] = df_precaution.apply(combine_precautions, axis=1)
df_precaution = df_precaution.drop(columns=precaution_cols)
df_precaution = df_precaution.drop_duplicates(subset=['Disease'])


# --- C. Merge all files into a final DataFrame ---


# Merge symptoms (new column All_Symptoms) with descriptions
df_final = pd.merge(
    df_dataset,
    df_description,
    on='Disease',
    how='left'
)

# Merge Precautions (New Column: All_Precaution)
df_final = pd.merge(
    df_final,
    df_precaution,
    on='Disease',
    how='left'
)

print(" Data merged and processed successfully. ✅")
print(f"Number of unique documents ready for indexing: {len(df_final)}")
print("\nExample of the first merged document:")
print(df_final.iloc[0].to_dict())


# --- D. Converting the final DataFrame to LangChain documents for RAG ---


def df_to_langchain_docs(df: pd.DataFrame) -> List[Document]:
    langchain_docs: List[Document] = []

    for index, row in df.iterrows():
       # Building the textual content of the document

        doc_content = f"""
        **Medical Case Knowledge**
        Disease: {row['Disease']}
        Symptoms: {row['All_Symptoms']}
        Description: {row['Description']}
        Precaution: {row['All_Precaution']}
        """

# Create a LangChain document. Each row in df_final becomes a document.
        langchain_docs.append(
            Document(
                page_content=doc_content.strip(),
                metadata={"source": "medical_dataset", "disease": row['Disease']}
            )
        )
    return langchain_docs

final_rag_docs = df_to_langchain_docs(df_final)


# 2. Create Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 3. Create Vector Store (the Retriever)
vectorstore = FAISS.from_documents(documents=final_rag_docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) #Bring the top 3documents

print("✅ Knowledge base is ready.")
print(f"   -> Loaded {len(final_rag_docs)} unique disease entries.")
print(f"   -> FAISS vector store created.")

Loading, processing, and preparing medical knowledge base from CSVs...
 Data merged and processed successfully. ✅
Number of unique documents ready for indexing: 304

Example of the first merged document:
{'Disease': 'Fungal infection', 'All_Symptoms': 'dischromic  patches, nodal skin eruptions, skin rash, itching', 'Description': 'In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, water, and plants. There are also some fungi that live naturally in the human body. Like many microbes, there are helpful fungi and harmful fungi.', 'All_Precaution': 'bath twice | use detol or neem in bathing water | keep infected area dry | use clean cloths'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Knowledge base is ready.
   -> Loaded 304 unique disease entries.
   -> FAISS vector store created.


### Define Generator LLM and Graph State

In [ ]:
# Code Cell 3: Define Generator LLM and Graph State
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from typing import List, Dict
from typing_extensions import TypedDict

# 1. Load the Generator LLM (DeepSeek)
# We'll load it in 4-bit precision to save memory in Colab!
# model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the tokenizer and model with quantization
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Use the BitsAndBytesConfig
    device_map="auto" # Automatically use the GPU if available
)

# Create a Hugging Face Pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512 # Set a limit on the answer length
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print(f"✅ Generator LLM ({model_id}) loaded.")

# 2. Define the Graph State
# This is the 'memory' of our agent as it runs.
# It holds the question, the documents we find, and the final answer.
class GraphState(TypedDict):
    """Represents the state of our graph.

    Attributes:
        question: The user's question
        documents: The list of retrieved documents
        generation: The LLM's final answer
        question_history: A list of all questions asked, to prevent loops
    """
    question: str
    documents: List[str]
    generation: str
    question_history: List[str] # Added for the exercise to prevent loops

print("✅ GraphState defined.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Generator LLM (microsoft/Phi-3-mini-4k-instruct) loaded.
✅ GraphState defined.


### Define Graph Nodes

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List
from typing_extensions import TypedDict

print("Defining graph nodes...")

# --- NODE 1: RETRIEVE ---
def retrieve_node(state: TypedDict) -> TypedDict:
    """Takes the question and retrieves documents (Medical Cases)."""
    print("---NODE: RETRIEVE ")
    question = state["question"]
    # Retrieve top documents using the retriever defined in Code Cell 2
    documents = retriever.invoke(question)
    doc_texts = [doc.page_content for doc in documents]

    return {
        "documents": doc_texts,
        "question": question,
        "question_history": state.get("question_history", [])
    }

# --- NODE 2: GRADE DOCUMENTS ---
def grade_documents_node(state: TypedDict) -> TypedDict:
    """Checks if the retrieved documents are relevant to the medical question."""
    print("---NODE: GRADE DOCUMENTS ")
    question = state["question"]
    documents = state["documents"]

#A valid claim should contain only 'document' and 'question'
    prompt_template = """You are a medical grader. Your job is to check if a
    retrieved medical case (containing a disease and symptoms) is relevant to the user's question about symptoms.
    Respond with a *single word*: 'yes' if relevant, 'no' if not.

    Document (Medical Case): {document}
    User Question: {question}

    Answer (yes/no):"""
    prompt = PromptTemplate.from_template(prompt_template)
# Here only the required variables are passed: 'question' and 'document'
    grader_chain = prompt | llm | StrOutputParser()

    relevant_docs = []
    for doc in documents:
        try:
            result = grader_chain.invoke({"question": question, "document": doc})
            score = result.strip().lower()
            if "yes" in score:
                print("  -> Grader Decision: Relevant ")
                relevant_docs.append(doc)
            else:
                print("  -> Grader Decision: NOT Relevant")
        except Exception as e:
            print(f"  -> Grader: Error - {e}")
            continue

    return {
        "documents": relevant_docs,
        "question": question,
        "question_history": state.get("question_history", [])
    }

# --- NODE 3: GENERATE ---
# --- NODE 3: GENERATE ---
def generate_node(state: TypedDict) -> TypedDict:
    """Takes the question and medical documents, and generates a diagnosis and precautions."""
    print("---NODE: GENERATE ---")
    question = state["question"]
    documents = state["documents"]

    # RAG prompt adapted for providing diagnosis and precautions
    prompt_template = """
You are a highly disciplined medical diagnosis assistant.
Your task is to analyze the 'Medical Context' and the user's 'Symptoms' to provide a diagnosis report.

**STRICT INSTRUCTIONS (MUST FOLLOW):**
1. Output your answer ONLY as a JSON object enclosed in triple backticks (```json ... ```).
2. DO NOT include any explanatory text, preambles, or disclaimers outside of the JSON block.
3. The JSON object MUST have the EXACT keys: "Disease", "Key_Symptoms" (as a list of strings), "Description" (as a string), "Precautions" (as a list of strings), and "Confidence_Level" (as a string).
4. Use ONLY the data provided in the 'Medical Context'. DO NOT hallucinate.
5. If no clear diagnosis is possible, set "Disease" to "Undetermined".

**CONFIDENCE LEVEL RULES (MUST FOLLOW):**
Determine the "Confidence_Level" based ONLY on how many symptoms from the user's input
match the symptoms associated with the disease in the Medical Context:

- If 3 or more symptoms match → "High"
- If exactly 2 symptoms match → "Medium"
- If exactly 1 symptom matches → "Low"
- If 0 symptoms match → "Very Low"

Symptoms: {question}
Medical Context: {context}

JSON Report:
"""

    prompt = PromptTemplate.from_template(prompt_template)
    # llm is assumed to be defined globally
    rag_chain = prompt | llm | StrOutputParser()

    raw_generation = rag_chain.invoke({"context": "\n".join(documents), "question": question})

    import re
    import json

    match = re.search(r'```json\s*(\{.*?\}\s*)```', raw_generation, re.DOTALL)

    if match:
        json_string = match.group(1).strip()
    else:
        match = re.search(r'(\{.*?\}\s*)', raw_generation, re.DOTALL)
        if match:
            json_string = match.group(1).strip()
        else:
            final_generation = raw_generation
            print("  -> Error: Could not find valid JSON. Returning raw output.")
            return {
                "documents": documents,
                "question": question,
                "generation": final_generation,
                "question_history": state.get("question_history", [])
            }

    try:
        report = json.loads(json_string)
        final_generation = "```json\n" + json.dumps(report, indent=4) + "\n```"
    except Exception as e:
        final_generation = raw_generation
        print(f"  -> Error: JSON parsing failed after extraction: {e}")

    return {
        "documents": documents,
        "question": question,
        "generation": final_generation,
        "question_history": state.get("question_history", [])
    }

# --- NODE 4: REWRITE ---
def rewrite_node(state: TypedDict) -> TypedDict:
    """Rewrites the question for a better search query, with loop prevention."""
    print("---NODE: REWRITE QUERY ---")
    question = state["question"]
    question_history = state.get("question_history", [])

    # Loop Prevention check
    if question in question_history:
        print("  -> Detected loop: Ending.")
        return {
            "documents": [],
            "generation": "Could not find relevant information after multiple attempts.",
            "question_history": question_history
        }

    question_history.append(question)

    # Rewrite prompt adapted for symptom queries
    prompt_template = """You are a query rewriter. Rewrite the following question (which is a list of symptoms) to be
    a concise and specific search query for a medical vector database.
    Respond ONLY with the rewritten query, nothing else.

    Original Question: {question}

    Rewritten Query:"""
    prompt = PromptTemplate.from_template(prompt_template)
    rewrite_chain = prompt | llm | StrOutputParser()

    new_question = rewrite_chain.invoke({"question": question}).strip()
    print(f"  -> Rewritten question: {new_question}")

    return {
        "question": new_question,
        "question_history": question_history,
        "documents": []
    }

print("✅ All nodes defined.")

Defining graph nodes...
✅ All nodes defined.


### Define Graph Edges and Compile

In [ ]:
# Code Cell 5: Define Graph Edges and Compile
from langgraph.graph import END, StateGraph
from typing import List, Dict
from typing_extensions import TypedDict

print("Defining conditional logic...")

def decide_to_generate_or_rewrite(state: TypedDict) -> str:
    """Decides whether to generate an answer, rewrite the query, or end."""
    print("---CONDITIONAL EDGE ")
    documents = state["documents"]
    question_history = state.get("question_history", [])

    # Check for loop prevention (End if generation already occurred)
    if state.get("generation"):
        print("  -> Decision: Loop detected or generation already done. END.")
        return "end"

    # 1. If we have relevant documents, generate
    if len(documents) > 0:
        print("  -> Decision: Relevant documents found. GENERATE.")
        return "generate"
    else:
        # 2. If no documents, check rewrite limit
        if len(question_history) >= 2: # 1 original + 2 rewrites = 3 attempts total
            print("  -> Decision: Max rewrites reached. END.")
            return "end" # Give up
        else:
            # 3. If no documents and limit not reached, rewrite
            print("  -> Decision: No relevant documents. REWRITE.")
            return "rewrite"

print("✅ Conditional logic defined.")

Defining conditional logic...
✅ Conditional logic defined.


In [ ]:
# Code Cell 6: Build and Compile the Graph
from langgraph.graph import END, StateGraph

print("Building the graph...")

# 1. Create the StateGraph
workflow = StateGraph(GraphState)

# 2. Add all 4 nodes
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("generate", generate_node)
workflow.add_node("rewrite", rewrite_node)

# 3. Set the entry point
workflow.set_entry_point("retrieve")

# 4. Add the standard edges
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("rewrite", "retrieve") # The loop
workflow.add_edge("generate", END)

# 5. Add the conditional edge
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate_or_rewrite,
    {
        "generate": "generate",
        "rewrite": "rewrite",
        "end": END
    }
)

# 6. Compile the graph
app = workflow.compile()
print("✅ LangGraph compiled and ready to run.")

Building the graph...
✅ LangGraph compiled and ready to run.


### Running the Final Agent

In [ ]:
# Code Cell 7: Run Your Agent!

print("--- 🚀 Test 1: Running Medical RAG Agent (Good Query) ---")
# Define a medical query based on symptoms in your dataset
query_good = "I have high fever, joint pain, and a runny nose. What is my diagnosis?"

inputs_good = {"question": query_good, "documents": [], "generation": "", "question_history": []}

print(f"\nUser Query: {query_good}\n")
# Run app.stream() and print the final 'generate' output
for output in app.stream(inputs_good):
    for key, value in output.items():
        print(f"\n--- Output from Node: {key} ---")
        if key == "generate":
            # Print the final answer clearly
            print("   -> 🤖 **Final Medical Diagnosis:**\n" + value['generation'])

print("\n--- 🏁 Run Complete ---")





--- 🚀 Test 1: Running Medical RAG Agent (Good Query) ---

User Query: I have high fever, joint pain, and a runny nose. What is my diagnosis?

---NODE: RETRIEVE 

--- Output from Node: retrieve ---
---NODE: GRADE DOCUMENTS 
  -> Grader Decision: Relevant 
  -> Grader Decision: Relevant 
  -> Grader Decision: Relevant 
---CONDITIONAL EDGE 
  -> Decision: Relevant documents found. GENERATE.

--- Output from Node: grade_documents ---
---NODE: GENERATE ---

--- Output from Node: generate ---
   -> 🤖 **Final Medical Diagnosis:**
```json
{
    "Disease": "Common Cold",
    "Key_Symptoms": [
        "High fever",
        "Joint pain",
        "Runny nose"
    ],
    "Description": "The common cold is a viral infection of your nose and throat (upper respiratory tract). It's usually harmless, although it might not feel that way. Many types of viruses can cause a common cold. Precautions include drinking vitamin C rich drinks, taking vapour, avoiding cold food, and keeping fever in check.",
    "

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app.py

import streamlit as st
import os
import pandas as pd
import torch
import json
import re
from typing import List
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# ------------------ CACHING FOR STABILITY ------------------

@st.cache_resource
def create_embedding_model():
    return HuggingFaceEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={"device": "cpu"}
    )

@st.cache_resource
def create_vectorstore(_docs):
    embedding_model = create_embedding_model()
    retriever = FAISS.from_documents(_docs, embedding=embedding_model)
    return retriever.as_retriever(search_kwargs={"k":3})

@st.cache_resource
def create_llm():
    model_id = "microsoft/Phi-3-mini-4k-instruct"
    # model_id="deepseek-ai/deepseek-coder-1.3b-instruct"

    # 4-bit quantization settings for GPU
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        low_cpu_mem_usage=True
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.1
    )
    return HuggingFacePipeline(pipeline=pipe)

# ------------------ LOAD & PROCESS DATA ------------------
data_path = '/content/drive/MyDrive/dataset.csv'
precaution_path = '/content/drive/MyDrive/symptom_precaution.csv'
description_path = '/content/drive/MyDrive/symptom_Description.csv'

if not os.path.exists(data_path):
    st.error("❌ Data files not found. Please ensure Google Drive is mounted and files are in the correct path.")
    st.stop()

try:
    df_dataset = pd.read_csv(data_path)
    df_precaution = pd.read_csv(precaution_path)
    df_description = pd.read_csv(description_path)
except Exception as e:
    st.error(f"❌ Error loading CSV files: {e}")
    st.stop()

symptom_cols = [c for c in df_dataset.columns if c.startswith('Symptom_')]
df_dataset[symptom_cols] = df_dataset[symptom_cols].fillna('')

def combine_symptoms(row):
    symptoms = [s.strip().replace('_',' ') for s in row[symptom_cols] if s.strip()]
    return ', '.join(list(set(symptoms)))

df_dataset['All_Symptoms'] = df_dataset.apply(combine_symptoms, axis=1)
df_dataset = df_dataset.drop(columns=symptom_cols).drop_duplicates(subset=['Disease']).reset_index(drop=True)

prec_cols = [c for c in df_precaution.columns if c.startswith('Precaution_')]
df_precaution[prec_cols] = df_precaution[prec_cols].fillna('No Further Precaution Mentioned')

def combine_precautions(row):
    precautions = [p.strip() for p in row[prec_cols] if p.strip() != 'No Further Precaution Mentioned']
    return ' | '.join(precautions)

df_precaution['All_Precaution'] = df_precaution.apply(combine_precautions, axis=1)
df_precaution = df_precaution.drop(columns=prec_cols).drop_duplicates(subset=['Disease'])

df_final = pd.merge(df_dataset, df_description, on='Disease', how='left')
df_final = pd.merge(df_final, df_precaution, on='Disease', how='left')

def df_to_docs(df):
    docs = []
    for _, row in df.iterrows():
        content = f"""
        Medical Case Knowledge
        Disease: {row['Disease']}
        Symptoms: {row['All_Symptoms']}
        Description: {row['Description']}
        Precaution: {row['All_Precaution']}
        """
        docs.append(Document(page_content=content.strip(), metadata={'source':'medical_dataset','disease':row['Disease']}))
    return docs

final_rag_docs = df_to_docs(df_final)

retriever = create_vectorstore(final_rag_docs)
llm = create_llm()

# ------------------ STREAMLIT APP ------------------
st.set_page_config(page_title="Medical RAG Agent", layout="wide")
st.title("🩺 Medical RAG Diagnosis Agent")

if 'history' not in st.session_state:
    st.session_state.history = []

# Sidebar UI
with st.sidebar:
    st.header("Upload Logo")
    uploaded_logo = st.file_uploader("Upload your logo (optional)", type=['png','jpg','jpeg'])
    if uploaded_logo:
        st.image(uploaded_logo, width=150)

    st.header("History Controls")
    if st.button("Clear History"):
        st.session_state.history = []
        st.rerun()

user_input = st.text_area("Enter your symptoms:", height=150)

if st.button("Send"):
    if user_input.strip():
        with st.spinner("Analyzing symptoms and generating diagnosis..."):
            # 1. Retrieval
            docs = retriever.invoke(user_input)
            context = '\n'.join([d.page_content for d in docs])

            # 2. Generation Prompt
            prompt_template = """
            You are a highly disciplined medical diagnosis assistant.
            Your task is to analyze the 'Context' and the user's 'Symptoms' to provide a diagnosis report.

            **STRICT INSTRUCTIONS (MUST FOLLOW):**
            1. Output your answer ONLY as a JSON object enclosed in triple backticks (```json ... ```).
            2. DO NOT include ANY explanatory text or instructions outside of the JSON block.
            3. The JSON object MUST have the EXACT keys: "Disease", "Key_Symptoms" (as a list of strings), "Description" (as a string), "Precautions" (as a list of strings), and "Confidence_Level" (as a string).
            4. Fill ALL fields with the ACTUAL VALUES from the context.
            5. Use ONLY the data provided in the 'Context'.
            6. If no clear diagnosis is possible, set "Disease" to "Undetermined".

            **CONFIDENCE LEVEL RULES (MUST FOLLOW):**
            Determine the "Confidence_Level" based ONLY on how many symptoms from the user's input
            match the symptoms associated with the disease in the Medical Context:

            - If 3 or more symptoms match → "High"
            - If exactly 2 symptoms match → "Medium"
            - If exactly 1 symptom matches → "Low"
            - If 0 symptoms match → "Very Low"

            Symptoms: {question}
            Context: {context}

            JSON Report:
            """

            prompt = PromptTemplate.from_template(prompt_template)
            rag_chain = prompt | llm | StrOutputParser()
            generation = rag_chain.invoke({'question': user_input, 'context': context})

        try:
            # 3. JSON Parsing and Cleanup

            match = re.search(r'```json\s*(\{.*?\}\s*)```', generation, re.DOTALL)

            if match:
                json_string = match.group(1).strip()
            else:
                match = re.search(r'(\{.*?\}\s*)', generation, re.DOTALL)
                if match:
                    json_string = match.group(1).strip()
                else:
                    raise ValueError("Could not find a valid JSON object structure in the model output.")

            json_string = json_string.replace('```json', '').replace('```', '').strip()

            # Attempt to load the extracted string
            report = json.loads(json_string)
            st.success("✅ Diagnosis completed.")

        except Exception as e:
            report = {'Error':f'Exception parsing JSON: {e}', 'Raw_Output': generation}
            st.error("❌ Error during JSON parsing. This is likely due to the LLM's output format.")

        st.session_state.history.append((user_input, report))
    else:
        st.warning("Please enter symptoms before sending.")

# Display Diagnosis History
st.header("Diagnosis History")
for q, r in st.session_state.history:
    with st.chat_message('user'):
        st.markdown(q)
    with st.chat_message('assistant'):
        if 'Error' in r:
            st.error(f"Error: {r['Error']}")
            if 'Raw_Output' in r:
                st.subheader("Raw Model Output (for debugging):")
                st.code(r['Raw_Output'])
        else:
            st.success(f"**Diagnosis:** {r.get('Disease', 'Undetermined')}")

            st.subheader("📋 Symptoms Provided")
            key_symptoms = r.get("Key_Symptoms")
            if isinstance(key_symptoms, list):
                st.markdown("- " + "  \n- ".join(key_symptoms))
            else:
                 st.markdown(f"**Symptoms:** {key_symptoms}")


            st.subheader("💡 Condition Description")
            st.markdown(r.get('Description', 'No description available from the knowledge base.'))

            st.subheader("⚠️ Precautionary Measures")
            precautions = r.get("Precautions")
            if isinstance(precautions, list):
                st.markdown("* " + "  \n* ".join(precautions))
            else:
                 st.markdown(f"**Precautions:** {precautions}")

            st.caption(f"Confidence Level: **{r.get('Confidence_Level', 'N/A')}**")


# Download button
if st.session_state.history:
    all_reports = [r for _, r in st.session_state.history]
    st.download_button("Download Reports JSON", data=json.dumps(all_reports, indent=2), file_name="diagnosis_reports.json")

print("✅ app.py file written successfully with robust JSON parsing and display logic.")

Writing app.py


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

print("Stopping all active ngrok tunnels...")
ngrok.kill()
print("All tunnels stopped. Ready for a fresh start.")

Stopping all active ngrok tunnels...
All tunnels stopped. Ready for a fresh start.


In [ ]:
# !pip install pyngrok
!pip install streamlit
from pyngrok import ngrok
import os
import time
import subprocess

NGROK_AUTH_TOKEN = "33SJ0eoJlLHPEV61oqyWpLJraBP_28LFemjgc8m5uFXCKyqtL"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("🚀 Starting Streamlit app...")
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(15)

print("Creating Ngrok tunnel...")
public_url = ngrok.connect(addr="8501")

print(f"--- ✅ Deployment Successful! ---")
print(f"Public URL for your app is: {public_url}")
print("Click the link above to access your Medical RAG Agent!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.1 MB/s eta 0:00:00
🚀 Starting Streamlit app...
Creating Ngrok tunnel...
--- ✅ Deployment Successful! ---
Public URL for your app is: NgrokTunnel: "https://faintish-herbaceously-elva.ngrok-free.dev" -> "http://localhost:8501"
Click the link above to access your Medical RAG Agent!
